In [1]:
import numpy as np
import cv2 as cv
import os
import json
from PIL import Image
import matplotlib.pyplot as plt
import time

%load_ext autoreload
%autoreload 2

In [2]:
from lac.perception.segmentation import Segmentation

In [11]:
segmentation = Segmentation()

In [ ]:
data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/hazard_2")
# data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/data_collection_1")

i = 1001
I1 = cv.imread(os.path.join(data_path, "front_left", f"{i}.png"), cv.IMREAD_GRAYSCALE)
image_seg_in = Image.open(os.path.join(data_path, "front_left", f"{i}.png")).convert("RGB")
plt.imshow(I1, cmap="gray")
plt.show()

In [ ]:
results, mask = segmentation.segment_rocks(image_seg_in)

In [46]:
from lac.util import color_mask

In [43]:
# Create Red, Green, Blue channels (only Red has values)
r = mask[:, :, 0]
g = np.zeros_like(mask[:, :, 0])
b = np.zeros_like(mask[:, :, 0])

# Merge channels into a color image
mask_colored = cv.merge([r, g, b])

In [56]:
mask_colored = color_mask(mask, (1, 0, 0))

In [ ]:
mask_colored.shape

In [ ]:
# Plot the original image with the mask overlaid as 50% transparency purple
plt.imshow(I1, cmap="gray")
plt.imshow(mask_colored, alpha=0.5)
plt.show()

In [8]:
from transformers import pipeline
import torch

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
checkpoint = "depth-anything/Depth-Anything-V2-base-hf"
pipe = pipeline("depth-estimation", model=checkpoint, device=device)

In [ ]:
i = 100

image_path = os.path.expanduser(
    "~/LunarAutonomyChallenge/output/data_collection_1/front_left/{}.png".format(i)
)
image_depth_in = Image.open(image_path)

predictions = pipe(image_depth_in)

# Plot image and predicted depth side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 10), gridspec_kw={"wspace": 0, "hspace": 0})
axes[0].imshow(image_depth_in, cmap="gray")
axes[1].imshow(predictions["depth"], cmap="gray")
for ax in axes:
    ax.axis("off")
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

In [ ]:
image = Image.open(os.path.join(data_path, "front_left", f"{i}.png"))
predictions = pipe(image)

# Plot image and predicted depth side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 10), gridspec_kw={"wspace": 0, "hspace": 0})
axes[0].imshow(image, cmap="gray")
axes[1].imshow(predictions["depth"], cmap="gray")
for ax in axes:
    ax.axis("off")
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()